In [1]:
import os
import cv2
import numpy as np
import json
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DATA_PATH = os.path.join(os.getcwd(), '../data/3_pts')
print(DATA_PATH)
PROCESSED_DATA_PATH = os.path.join(os.getcwd(), '../processed/3_pts')
if not os.path.exists(PROCESSED_DATA_PATH):
    os.makedirs(PROCESSED_DATA_PATH)
if not os.path.exists(os.path.join(PROCESSED_DATA_PATH,'images')):
    os.makedirs(os.path.join(PROCESSED_DATA_PATH,'images'))

/home/boomalope/nerf_estimator/src/nerf_estimator/nerf_vision/scripts/../data/3_pts


In [3]:
#Gazebo Param: 
#Definition: https://github.com/ros-simulation/gazebo_ros_pkgs/blob/398cde933cba5d72921055d57d4faa47c4445654/gazebo_plugins/src/gazebo_ros_camera_utils.cpp#L524

In [4]:
#Camera parameters
#Definitions: https://docs.nerf.studio/quickstart/data_conventions.html

fl_x = 762.72 # focal length in x
fl_y = 762.72# focal length in y
w = 1280 # image width
h = 720 # image height
cx = w//2 # optical center in x
cy = h//2 # optical center in y
camera_model = "OPENCV" # camera model
k1 = 0 # radial distortion coefficient k1
k2 = 0 # radial distortion coefficient k2
k3 = 0 # radial distortion coefficient k3
p1 = 0 # tangential distortion coefficient p1
p2 = 0 # tangential distortion coefficient p2

In [5]:
def from_tf_to_matrix(tf):
    #tf is the frame of the robot relative to the camera
    xyz = tf[:3]
    q = tf[3:7]
    goal = tf[7:]

    #Rotation matrix
    r = Rotation.from_quat(q)
    r = r.as_matrix()

    #Additional rotation  matrix to shift camera from +x to -z direction and make the x axis point to the right
    A = Rotation.from_euler('YZX', [-90, -90, 0], degrees=True).as_matrix()

    #Translation matrix in the world frame
    t = np.zeros((4,4))
    t[:3,:3] = np.matmul(r.T,A)
    t[:3,3] = -np.matmul(r.T,xyz)
    t[3,3] = 1

    print(t[:3,3])
    print(Rotation.from_matrix(t[:3,:3]).as_euler('xyz', degrees=True))
    
    return t

In [6]:
data_cnt = 0
frames = []
while True:
    #Get data paths
    data_img = os.path.join(DATA_PATH, '{}.png'.format(data_cnt))
    data_tf= os.path.join(DATA_PATH, '{}.txt'.format(data_cnt))

    if not os.path.exists(data_img):
        break
    
    #Get tf as np array
    tf = np.loadtxt(data_tf, delimiter=',')


    file_path = "images/{}.png".format(data_cnt)
    transform_matrix = from_tf_to_matrix(tf)

    #Append to frames
    frames.append({
        "file_path": file_path,
        "transform_matrix": transform_matrix.tolist(),
    })

    #Save image in processed folder
    img = cv2.imread(data_img)
    cv2.imwrite(os.path.join(PROCESSED_DATA_PATH, file_path), img)
    
    data_cnt += 1       

[-2.25642654e+00  1.13682232e-02  2.18303093e-04]
[ 9.00408338e+01  8.18179440e-03 -9.08536107e+01]
[-2.25697250e+00  1.12226500e-02  1.13397671e-04]
[ 9.00434892e+01  8.95643961e-03 -9.08499146e+01]
[-2.25699480e+00  5.03875846e-02  2.82219489e-04]
[ 9.00392195e+01  7.41760638e-03 -9.18441318e+01]
[-2.24093878e+00  2.77171619e-01  3.90588670e-04]
[ 9.00364571e+01  8.78008071e-03 -9.76169148e+01]
[-2.19164288e+00  5.47553525e-01  2.04386286e-04]
[ 9.00411569e+01  1.15013263e-02 -1.04596659e+02]
[-2.14828662e+00  6.98441727e-01  4.47930531e-04]
[ 9.00349278e+01  1.66922625e-02 -1.08579013e+02]
[-2.10245707e+00  8.26125799e-01 -2.66277753e-05]
[ 9.00468988e+01  2.33669638e-02 -1.12020029e+02]
[-2.01153191e+00  1.02753281e+00  3.31696428e-04]
[ 9.00378171e+01  2.26145104e-02 -1.17625452e+02]
[-1.91270023e+00  1.20128605e+00  7.43203565e-04]
[ 9.00273508e+01  2.53377800e-02 -1.22695857e+02]
[-1.79986345e+00  1.36475973e+00  5.81460107e-04]
[ 9.00314069e+01  3.01326977e-02 -1.27738977e+02]


In [7]:
transform_dictionary = {}
transform_dictionary["fl_x"] = fl_x
transform_dictionary["fl_y"] = fl_y
transform_dictionary["w"] = w
transform_dictionary["h"] = h
transform_dictionary["cx"] = cx
transform_dictionary["cy"] = cy
transform_dictionary["k1"] = k1
transform_dictionary["k2"] = k2
transform_dictionary["k3"] = k3
transform_dictionary["p1"] = p1
transform_dictionary["p2"] = p2
transform_dictionary["camera_model"] = camera_model
transform_dictionary["frames"] = frames

#Save as json 
with open(os.path.join(PROCESSED_DATA_PATH, "transforms.json"), 'w') as outfile:
    json.dump(transform_dictionary, outfile)
